### Vector_auto_regression 

In [24]:
import glob
import pandas as pd
import numpy as np
import requests
#import html5lib
from bs4 import BeautifulSoup
import datetime
import re


tweets_path = 'Tweets/bitcoin/'
crypto_name = 'bitcoin' 
dates=[]

In [25]:
cols = ['date','polarity']
lst = []
df1 = pd.DataFrame(lst, columns=cols)
lst_dict = []
time_pol={}  # dict of date and polarity.

for f in glob.glob( str(tweets_path) + crypto_name + '*.txt'): # f is file name.

    df_datawise = pd.read_csv(f, sep='\t', encoding='utf-8', parse_dates=True)
    polarity = 0.00

    ct=0;
    for idx,row in df_datawise.iterrows():
        polarity += float(row["polarity"])
        dt=row["date"]
        ct=ct+1
    avg_pol=float(polarity/ct)
    lst_dict.append({'date':dt, 'polarity':avg_pol,})
    time_pol[dt]=avg_pol

            
df4 = df1.append(lst_dict) # avg polarity dataframe

In [26]:
df_price=pd.read_csv("bitcoin.csv", encoding='utf-8', parse_dates=True)


In [27]:
df_price['Timestamp'].size


525600

In [28]:
cols = ['Timestamp','Close']
lst = []
df_price1 = pd.DataFrame(lst, columns=cols)
lst_dict1 = []

In [29]:
df_price['Timestamp']=pd.to_datetime(df_price['Timestamp'])
for idx,r in df_price.iterrows():
    dt = r["Timestamp"].strftime("%Y-%m-%d")
#     print(time)
#     if(dt=="2017-02-11"):
#         print(dt)
    if dt in time_pol.keys():
        lst_dict1.append({'Timestamp':r['Timestamp'], 'Close':r['Close'],})
#         print(dt)
df_price_new = df_price1.append(lst_dict1)  #dataframe of 91 days

In [30]:
df_price_new

,Timestamp,Close
0,2017-09-01 00:00:00,4740.00
1,2017-09-01 00:01:00,4740.00
2,2017-09-01 00:02:00,4739.90
3,2017-09-01 00:03:00,4738.59
4,2017-09-01 00:04:00,4736.09
...,...,...
131035,2017-11-30 23:55:00,9800.01
131036,2017-11-30 23:56:00,9775.99
131037,2017-11-30 23:57:00,9751.06
131038,2017-11-30 23:58:00,9769.99


In [31]:
price_arr=df_price_new.to_numpy()

In [32]:
price_arr.shape


(131040, 2)

### creating data frame

In [33]:
cols = ['Timestamp','Polarity','Price']
lst = []
df_price_final = pd.DataFrame(lst, columns=cols)
lst_dict2 = []

for i in range(1,62):
    cols.append(str(i))
    


i=0
while (i< price_arr.shape[0]):
#     print(price_arr[i,0])
    dt = price_arr[i,0].strftime("%Y-%m-%d")
    polarity=time_pol[dt]
    dict_63={}
    price=0
    while(i< price_arr.shape[0] and dt==price_arr[i,0].strftime("%Y-%m-%d")):
        price=price+price_arr[i,1]
        i=i+1
        ct=ct+1
        
    avg_pr=price/ct
    dict_63['Timestamp']=dt
    dict_63['Polarity']=polarity
    dict_63['Price']=avg_pr
    lst_dict2.append(dict_63)
    
    
    

        
df_final=df_price_final.append(lst_dict2)

In [34]:
df_final

,Timestamp,Polarity,Price
0,2017-09-01,0.095204,3307.658505
1,2017-09-02,0.085108,1955.923156
2,2017-09-03,0.087569,1348.163036
3,2017-09-04,0.087290,1014.192134
4,2017-09-05,0.100801,808.729080
...,...,...,...
86,2017-11-26,0.116530,102.786000
87,2017-11-27,0.115579,108.765809
88,2017-11-28,0.102703,109.812027
89,2017-11-29,0.093324,115.923256


In [36]:
df_final.to_csv(r'day_avg.csv')

## Augmented Dickey-Fuller Test (ADF Test)
### Price:
P value for price is 0.015<.05. Hence we can neglect Null Hypothesis and price feature's time series is stationary.

In [38]:

from pandas import read_csv
from statsmodels.tsa.stattools import adfuller
# series = read_csv('daily-total-female-births.csv', header=0, index_col=0, squeeze=True)
# X = series.values
result = adfuller(df_final['Price'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))
    


ADF Statistic: -3.283814
p-value: 0.015618
Critical Values:
	1%: -3.515
	5%: -2.898
	10%: -2.586


### Polarity:
P value for price is 0<.05. Hence we can neglect Null Hypothesis and price feature's time series is stationary.

In [40]:
from pandas import read_csv
from statsmodels.tsa.stattools import adfuller
# series = read_csv('daily-total-female-births.csv', header=0, index_col=0, squeeze=True)
# X = series.values
result = adfuller(df_final['Polarity'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))
    


ADF Statistic: -6.080220
p-value: 0.000000
Critical Values:
	1%: -3.505
	5%: -2.894
	10%: -2.584


In [46]:
df_final

,Timestamp,Polarity,Price
0,2017-09-01,0.095204,3307.658505
1,2017-09-02,0.085108,1955.923156
2,2017-09-03,0.087569,1348.163036
3,2017-09-04,0.087290,1014.192134
4,2017-09-05,0.100801,808.729080
...,...,...,...
86,2017-11-26,0.116530,102.786000
87,2017-11-27,0.115579,108.765809
88,2017-11-28,0.102703,109.812027
89,2017-11-29,0.093324,115.923256


### VAR daywise prediction

In [116]:
x=df_final.drop(columns=['Timestamp'])

In [117]:
data=x.to_numpy()

In [194]:
train = data[:int(0.9*(len(data)))]
valid = data[int(0.1*(len(data))):]

#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

/home/himanshu/.local/lib/python3.8/site-packages/statsmodels/base/wrapper.py:34: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


In [195]:
valid.shape

(82, 2)

In [121]:
y_pred=[]
for i in range(prediction.shape[0]):
    y_pred.append(prediction[i][1])


In [122]:
y_act=[]
for i in range(valid.shape[0]):
    y_act.append(valid[i][1])

In [70]:
from sklearn.metrics import r2_score

In [72]:
r2_score(y_act, y_pred)

-37.71174274223873

### VAR minutewise prediction

In [125]:
df=pd.read_csv('time_frame.csv')


In [126]:
df1=x=df.drop(columns=['Timestamp','Unnamed: 0'])

In [127]:
df1.columns

Index(['polarity', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35',
       '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47',
       '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59',
       '60', '61'],
      dtype='object')

In [128]:
# split=int(0.8*df1.shape[0])
# train=df1.iloc[:split,:]
# test=df1.iloc[split:,:]
data_np=df1.to_numpy()


In [199]:
train = data_np[:int(0.9*(len(data_np)))]
valid = data_np[int(0.9*(len(data_np))):]

In [200]:
train.shape

(117882, 62)

In [201]:
valid.shape

(13098, 62)

In [202]:
from statsmodels.tsa.vector_ar.var_model import VAR

model1 = VAR(endog=train)
model_fit = model1.fit()


In [203]:
prediction1 = model_fit.forecast(model_fit.y, steps=len(valid))

In [204]:
prediction1.shape

(13098, 62)

In [214]:
import math

In [155]:
# valid[15000][61]

8249.99

In [250]:
y_pred=[]
for i in range(prediction1.shape[0]):
    y_pred.append(int(prediction1[i][61]))


In [253]:
y_act=[]
for i in range(prediction1.shape[0]):
    y_act.append(int(valid[i][61]))

In [256]:
r2_score(y_act,y_pred)

0.3161281351618659